In [54]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime
from splinter import Browser
from selenium.webdriver.common.keys import Keys
import time
import json
import os
import shutil

In [55]:
# Path to chromedriver.exe
executable_path = {'executable_path':'C:/Users/zacha/AppData/Local/bin/chromedriver.exe'}

# Open the Splinter Browswer
browser = Browser('chrome', **executable_path)

In [56]:
# URL to find information on Coronavirus
url1 = "https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83"

url2 = "https://dshs.texas.gov/coronavirus/"

url3 = "http://publichealth.harriscountytx.gov/Resources/2019-Novel-Coronavirus/Harris-County-COVID-19-Confirmed-Cases"

url4 = "https://www.khou.com/section/news/health/coronavirus"

In [57]:
# Visiting the url 
browser.visit(url1)

In [58]:
time.sleep(7)# Creating the Beautiful soup for URL
html = browser.html
soup = bs(html, 'html.parser')

In [59]:
# All the Texas counties
# counties = soup.find('nav', attrs={'class':'feature-list'})

# getting all the spans with the information for each Texas county
# rows = counties.find_all('span')




In [60]:
# counter = 0

# county = []
# cases = []

# for row in rows:
#     try:
#         print(rows[counter].text.strip().split('\n'))
#         county.append(rows[counter].text.strip().split('\n')[0])
#         cases.append(rows[counter].text.strip().split('\n')[1])
#         counter = counter + 3
        
#     except:
#         print('failure')

In [61]:
# df = pd.DataFrame(list(zip(county, cases)), columns =['Texas_County', 'COVID_19_Cases'])
# df

In [62]:
# Droping the word 'County'
# df['Texas_County'] = df['Texas_County'].apply(lambda x : x.split(' County')[0])

In [63]:
# Revise? 
reported_cases = soup.find_all('text')[6].text
reported_cases

'5,330'

In [64]:
deaths = soup.find_all('text')[8].text
deaths

'90'

In [65]:
df_stats = pd.DataFrame({'Cases':[reported_cases], 'Deaths':[deaths]})
df_stats.to_csv('static/csv/stats.csv', index=False)

In [66]:
# Getting Total Cases and Deaths 
browser.visit(url2)

In [67]:
browser.find_by_text('Accessible version').click()
time.sleep(5)

In [68]:
source = 'C:/Users/zacha/Downloads/'
destination = 'C:/Users/zacha/Documents/COVID-19/'
filename = 'CaseCountData.xlsx'

In [69]:
shutil.move(os.path.join(source, filename), os.path.join(destination, filename))

'C:/Users/zacha/Documents/COVID-19/CaseCountData.xlsx'

In [70]:
# dest = shutil.move(source, destination, )

In [71]:
df = pd.read_excel('CaseCountData.xlsx', header=1).dropna()
df.sort_values(by=['Positive'], ascending=False, inplace=True)
df['County'] = df.apply(lambda x: x['County'].rstrip(), axis=1)
df

,No.,County,Positive,Fatalities
57,58.0,Harris,955,8
29,30.0,Dallas,831,17
127,128.0,Travis,351,3
122,123.0,Tarrant,325,6
32,33.0,Denton,254,5
...,...,...,...,...
89,90.0,Martin,1,0
96,97.0,Montague,1,0
99,100.0,Morris,1,0
102,103.0,Newton,1,0


In [72]:
browser.visit(url3)

In [73]:
harris_table = browser.find_by_id('covidtable')

In [74]:
html_harris = browser.html
harris_soup = bs(html_harris, 'html.parser')

In [75]:
harris_data = []
table = harris_soup.find('table', attrs={'id':'covidtable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    harris_data.append([ele for ele in cols if ele]) # Get rid of empty values

In [76]:
columns = ['Patient', 'Sex', 'Age_Range', 'Location','Exposure', 'Status']


In [77]:
harris_data = harris_data[1:]

In [78]:
harris_df = pd.DataFrame(harris_data, columns=columns)

In [79]:
harris_df.sort_values(by=['Age_Range'], inplace=True)

In [80]:
harris_df

,Patient,Sex,Age_Range,Location,Exposure,Status
43,44,Male,0 - 9,NW,Possible Exposure to Case,Active
45,46,Female,0 - 9,NW,Possible Exposure to Case,Active
172,173,Female,0 - 9,NW,Possible Exposure to Case,Active
185,186,Female,0 - 9,SE,Possible Exposure to Case,Active
147,148,Male,10 - 19,SE,Possible Exposure to Case,Active
...,...,...,...,...,...,...
154,155,Female,80+,SE,Investigation Pending,Active
246,247,Female,80+,SW,Investigation Pending,Active
15,16,Male,80+,NW,No Known Exposure,Deceased
158,159,Female,80+,NW,History of Travel,Active


In [81]:
harris_df.to_csv('static/csv/harris_stats.csv', index=False, header=True)

In [82]:
harris_df.head()

,Patient,Sex,Age_Range,Location,Exposure,Status
43,44,Male,0 - 9,NW,Possible Exposure to Case,Active
45,46,Female,0 - 9,NW,Possible Exposure to Case,Active
172,173,Female,0 - 9,NW,Possible Exposure to Case,Active
185,186,Female,0 - 9,SE,Possible Exposure to Case,Active
147,148,Male,10 - 19,SE,Possible Exposure to Case,Active


In [83]:
All_Tex_Counties_df = pd.read_csv('static/csv/Texas_Counties.csv')
All_Tex_Counties_df.head()

,X (Lat),Y (Long),CNTY_NM,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,-97.492799,29.456415,Gonzales,90,48177,2.124911,0.257805,"(-97.492799, 29.456415)"
1,-98.697292,27.043405,Jim Hogg,125,48247,2.271751,0.267624,"(-98.697292, 27.043405)"
2,-97.681378,26.924094,Kenedy,66,48261,5.067864,0.389397,"(-97.681378, 26.924094)"
3,-96.965687,30.310651,Lee,144,48287,2.213853,0.153990,"(-96.965687, 30.310651)"
4,-95.853568,32.211881,Henderson,108,48213,3.152909,0.235056,"(-95.853568, 32.211881)"


In [84]:
All_Tex_Counties = All_Tex_Counties_df['CNTY_NM'].tolist()

In [85]:
df_counties = df['County'].tolist()

In [86]:
Counties_with_Cases = []

for i in df_counties:
    
    if i.rstrip() in All_Tex_Counties:
        Counties_with_Cases.append(i.rstrip())
        print(f'{i}---found')
        
    elif i == 'DeWitt':
        Counties_with_Cases.append('De Witt')
        print(f'{i}---found')
    else:
        print(f'Houston we have a problem: {i}')

Harris---found
Dallas---found
Travis---found
Tarrant---found
Denton---found
Bexar---found
Collin---found
Fort Bend---found
Galveston---found
Brazoria---found
Lubbock---found
Montgomery---found
Hidalgo---found
El Paso---found
Brazos---found
Webb---found
Williamson---found
Cameron---found
Smith---found
Bell---found
McLennan---found
Hays---found
Wichita---found
Nueces---found
Jefferson---found
Matagorda---found
Victoria---found
Ellis---found
Guadalupe---found
Potter---found
Randall---found
Midland---found
Comal---found
Johnson---found
Taylor---found
Washington---found
Hardin---found
Ector---found
Nacogdoches---found
Grayson---found
Chambers---found
Bowie---found
Tom Green---found
Rockwall---found
Fayette---found
Bastrop---found
Shelby---found
Castro---found
Wharton---found
Kaufman---found
Hunt---found
Calhoun---found
Hood---found
Angelina---found
Kendall---found
Gregg---found
Rusk---found
Hockley---found
Waller---found
Orange---found
Polk---found
Wilson---found
Hale---found
Moore---found


In [87]:
All_Tex_Counties_df = All_Tex_Counties_df.set_index('CNTY_NM')

In [88]:
Geo_df = All_Tex_Counties_df.loc[Counties_with_Cases]
Geo_df.head()

,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
CNTY_NM,,,,,,,
Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
Denton,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"


In [89]:
Geo_df = Geo_df.reset_index()
Geo_df = Geo_df.rename(columns={"CNTY_NM": "County"})
Geo_df

,County,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
1,Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
2,Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
4,Denton,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"
...,...,...,...,...,...,...,...,...
140,Martin,-101.951246,32.306058,156,48317,1.912684,0.227089,"(-101.951246, 32.306058)"
141,Montague,-97.724670,33.674963,169,48337,2.090266,0.235633,"(-97.72467, 33.674963)"
142,Morris,-94.732051,33.113685,172,48343,1.538698,0.064495,"(-94.732051, 33.113685)"
143,Newton,-93.744630,30.786200,176,48351,3.669102,0.229430,"(-93.74463, 30.7862)"


In [90]:
result = pd.merge(df, Geo_df, on='County')
result = result.rename(columns={'X (Lat)':'long', 'Y (Long)':'lat'})

result.head()

,No.,County,Positive,Fatalities,long,lat,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,58.0,Harris,955,8,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
1,30.0,Dallas,831,17,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
2,128.0,Travis,351,3,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,123.0,Tarrant,325,6,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
4,33.0,Denton,254,5,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"


In [91]:
result = result[['County', 'Positive', 'Fatalities', 'lat', 'long']]
result

,County,Positive,Fatalities,lat,long
0,Harris,955,8,29.859671,-95.397821
1,Dallas,831,17,32.766537,-96.777819
2,Travis,351,3,30.334233,-97.781947
3,Tarrant,325,6,32.771852,-97.291165
4,Denton,254,5,33.205574,-97.116812
...,...,...,...,...,...
139,Martin,1,0,32.306058,-101.951246
140,Montague,1,0,33.674963,-97.724670
141,Morris,1,0,33.113685,-94.732051
142,Newton,1,0,30.786200,-93.744630


In [92]:
def df_to_geojson(df, properties, lat='lat', lon='long'):
    prefix = 'var' + ' data' + ' = '
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point','coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lat],row[lon]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ['County', 'Positive', 'Fatalities']
javascript_file = df_to_geojson(result, cols)
# {'type': 'FeatureCollection', 'features': [{'geometry': {'type': 'Point', 'coordinates': [-70.5867, -29.9953, 760]}, 'type': 'Feature', 'properties': {'name': 'A', 'description': 'Place a'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.4933, -30.1217, 1250]}, 'type': 'Feature', 'properties': {'name': 'B', 'description': 'Place b'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.5008, -30.0953, 1185]}, 'type': 'Feature', 'properties': {'name': 'C', 'description': 'Place c'}}]}

In [93]:

with open('static/js/map.json','w') as f:
    json.dump(javascript_file, f, indent=4, sort_keys=False)

In [94]:
browser.visit(url4)

In [95]:
headlines_harris = browser.html
headline_soup = bs(headlines_harris, 'html.parser')

In [96]:
# lis = headline_soup.find_all('li', class_="headline-list-with-abstract__item")
links = headline_soup.find_all('div', class_="headline-list-with-abstract__headline")

titles = []
hrefs = []

In [97]:
recent = headline_soup.find_all('div', class_="story-snapshot-with-abstract__headline")[0]

In [98]:
recent

<div class="story-snapshot-with-abstract__headline">
<a class="story-snapshot-with-abstract__link" data-tracking="story-snapshot-with-abstract/headlineLinkClicked" href="/article/news/health/coronavirus/judge-hidalgo-amends-stay-home-order-to-allow-small-religious-gatherings-that-follow-cdc-guidelines/285-995c77a8-f43b-471d-92e2-89558fd8f221">
<div class="story-snapshot-with-abstract__desktop-title dot-ellipsis dot-height-180" style="overflow-wrap: break-word;">
                        Judge Hidalgo amends Stay Home order to allow small religious gatherings that follow CDC guidelines
                    </div>
<div class="story-snapshot-with-abstract__mobile-title dot-ellipsis dot-height-135" style="overflow-wrap: break-word;">
                        Judge Hidalgo amends Stay Home order to allow small religious gatherings that follow CDC guidelines
                    </div>
</a>
</div>

In [99]:
recent_title = recent.find('div', class_="story-snapshot-with-abstract__desktop-title dot-ellipsis dot-height-180").text.strip()
recent_title

'Judge Hidalgo amends Stay Home order to allow small religious gatherings that follow CDC guidelines'

In [100]:
titles.append(recent_title)
hrefs.append('https://www.khou.com' + recent.a['href'])

In [101]:
links

[<div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="headline-list-with-abstract/headlineLinkClicked" href="/article/news/health/coronavirus/coronavirus-updates-houston-live-blog-april-3/285-6d1d2743-5612-4773-8627-3b9622f845d8">
                     Coronavirus updates: Over 80 people test positive at Texas City nursing home
                 </a>
 </div>,
 <div class="headline-list-with-abstract__headline"> <a class="headline-list-with-abstract__link">4 Signs Your Heart Is Quietly Failing You</a> </div>,
 <div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="headline-list-with-abstract/headlineLinkClicked" href="/article/sports/mlb/astros/justin-verlander-donating-mlb-paychecks-to-organizations-helping-those-affected-by-covid-19-crisis/285-1d3b8197-7c20-4869-a8d2-40c876882265">
                     Justin Verlander donating MLB paychecks to organizations helping thos

In [102]:

for article in links:
    
    try:
        
        title = article.find('a', class_="headline-list-with-abstract__link").text
        
        link = article.a['href']
        
        if (title and link):
            
            
            print('-------------')
            print(title)
            print(link)
            titles.append(title.strip())
            hrefs.append('https://www.khou.com' + link)
            
    except:
        print('no')

-------------

                    Coronavirus updates: Over 80 people test positive at Texas City nursing home
                
/article/news/health/coronavirus/coronavirus-updates-houston-live-blog-april-3/285-6d1d2743-5612-4773-8627-3b9622f845d8
no
-------------

                    Justin Verlander donating MLB paychecks to organizations helping those affected by COVID-19 crisis
                
/article/sports/mlb/astros/justin-verlander-donating-mlb-paychecks-to-organizations-helping-those-affected-by-covid-19-crisis/285-1d3b8197-7c20-4869-a8d2-40c876882265
-------------

                    A mounting casualty of coronavirus crisis: Health care jobs
                
/article/news/health/coronavirus/health-care-jobs-are-a-mounting-casualty-of-coronavirus-crisis/285-d656ba85-b425-49c3-85b5-b325340e3a64
-------------

                    67 residents at San Antonio nursing home test positive, 1 death
                
/article/news/health/coronavirus/city-officials-provide-update-on

In [103]:
latest_headlines_df = pd.DataFrame(list(zip(titles, hrefs)), columns =['Headline', 'link'])

In [104]:
latest_headlines_df

,Headline,link
0,Judge Hidalgo amends Stay Home order to allow ...,https://www.khou.com/article/news/health/coron...
1,Coronavirus updates: Over 80 people test posit...,https://www.khou.com/article/news/health/coron...
2,Justin Verlander donating MLB paychecks to org...,https://www.khou.com/article/sports/mlb/astros...
3,A mounting casualty of coronavirus crisis: Hea...,https://www.khou.com/article/news/health/coron...
4,67 residents at San Antonio nursing home test ...,https://www.khou.com/article/news/health/coron...
5,Coronavirus live updates: Delta pilots union s...,https://www.khou.com/article/news/health/coron...
6,Asian Americans use social media to mobilize a...,https://www.khou.com/article/news/health/coron...


In [105]:
latest_headlines_df.to_csv('static/csv/headlines.csv', index=False, header=True)

In [106]:
browser.quit()